# 1 Import Packages

In [1]:
import xarray as xr
import numpy as np
import os
import datetime
import src.config as config
import src.utils as utils

import sklearn.linear_model
from tqdm import tqdm
import cartopy.crs as ccrs
import joblib

import matplotlib.pyplot as plt

# 2 Metadata

In [31]:
lev_index = 23
kfold_index = 0
exp_name = "cv"
datetime_string = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
problem_name = "streamfunction"

In [32]:
ml_transform_path = os.path.join(config.data_pro_path,"ml_transform", exp_name, problem_name)

In [33]:
import glob

In [34]:
train_x_filename = "train_data_stack_samplestandardized_lev_{}_{}.nc".format(lev_index, kfold_index)
valid_x_filename = "valid_data_stack_samplestandardized_lev_{}_{}.nc".format(lev_index, kfold_index)

train_y_filename = "train_data_amoc_depth_mask_lat_mask_samplestandardized_{}_{}.nc".format(lev_index, kfold_index)
valid_y_filename = "valid_data_amoc_depth_mask_lat_mask_samplestandardized_{}_{}.nc".format(lev_index, kfold_index)

In [35]:
model_name = "linear_level_{}_kfold_{}".format(lev_index, kfold_index)
model_path = os.path.join(config.model_path,exp_name, datetime_string,model_name)

model_plot_path = os.path.join(model_path, "plots")
os.makedirs(model_plot_path, exist_ok=True)

# 3 Load Data

In [36]:
train_x_xr = xr.open_dataset(os.path.join(ml_transform_path, train_x_filename))
valid_x_xr = xr.open_dataset(os.path.join(ml_transform_path, valid_x_filename))

In [37]:
train_y_xr = xr.open_dataset(os.path.join(ml_transform_path, train_y_filename))
valid_y_xr = xr.open_dataset(os.path.join(ml_transform_path, valid_y_filename))

train_data_amoc_depth_1020_lat_26_samplestd  = xr.load_dataset(os.path.join(ml_transform_path,"train_data_amoc_depth_1020_lat_26_samplestd.nc" ))
train_data_amoc_depth_1020_lat_26_samplemean = xr.load_dataset(os.path.join(ml_transform_path,"train_data_amoc_depth_1020_lat_26_samplemean,nc"))

# 4 Processing

In [38]:
train_x_xr_stack = train_x_xr.stack(sample=("realization","time")).stack(feature=("lon","lat"))
valid_x_xr_stack = valid_x_xr.stack(sample=("realization","time")).stack(feature=("lon","lat"))

feature_coords = train_x_xr_stack.feature
sample_coords = train_x_xr_stack.sample

train_x_xr_stack_dropna = train_x_xr_stack.dropna(dim="feature")
valid_x_xr_stack_dropna = valid_x_xr_stack.dropna(dim="feature")

feature_dropna_coords = train_x_xr_stack_dropna.feature

In [44]:
train_y_xr_stack = train_y_xr.stack(sample=("realization","time"), feature=("depth_2","lat"))
valid_y_xr_stack = valid_y_xr.stack(sample=("realization","time"), feature=("depth_2","lat"))

# 5 Model

In [45]:
valid_sample_coords = valid_y_xr_stack.sample
train_sample_coords = train_y_xr_stack.sample

feature_drop_coords = train_x_xr_stack_dropna.feature
feature_full_coords = train_x_xr_stack.feature 

train_x_ml_xr = train_x_xr_stack_dropna["rho"]
valid_x_ml_xr = valid_x_xr_stack_dropna["rho"]

train_y_ml_xr = train_y_xr_stack["atlantic_moc"]
valid_y_ml_xr = valid_y_xr_stack["atlantic_moc"]

train_y_true_ml_xr = train_y_xr_stack["atlantic_moc"]
valid_y_true_ml_xr = valid_y_xr_stack["atlantic_moc"]


In [ ]:
def mse_xr(pred, true, sample_dim = "sample"):
    return np.square(pred - true).mean(dim=sample_dim)

## 5.1 Train

In [ ]:
>>> from sklearn.multioutput import MultiOutputRegressor
>>> from sklearn.linear_model import Ridge

In [48]:
from sklearn.multioutput import MultiOutputRegressor

In [50]:
train_y_ml_xr

<xarray.DataArray 'atlantic_moc' (sample: 13600, feature: 1800)>
array([[-1.3657322 , -1.37474947, -1.38006602, ..., -0.40803638,
        -0.40911199, -0.66816309],
       [-1.38027346, -1.38902054, -1.39391435, ..., -0.50865885,
        -0.52482174, -0.67439533],
       [-1.36941597, -1.37907811, -1.38464767, ..., -0.63792336,
        -0.56662271, -0.74946996],
       ...,
       [-1.3781807 , -1.38606708, -1.39063634, ..., -0.73903469,
        -0.6726337 , -0.82693528],
       [-1.38281016, -1.39217647, -1.39764845, ..., -0.67279125,
        -0.70561808, -0.86862661],
       [-1.37863606, -1.38530997, -1.3892139 , ..., -0.61550171,
        -0.5609974 , -0.80669271]])
Coordinates:
    lon          float64 ...
  * sample       (sample) object MultiIndex
  * realization  (sample) object 'lkm0001' 'lkm0001' ... 'lkm0100' 'lkm0100'
  * time         (sample) datetime64[ns] 1870-07-16T11:15:00 ... 2005-07-16T1...
  * feature      (feature) object MultiIndex
  * depth_2      (feature) float64 0.0 0.0 0.0 ... 1.98e+03 1.98e+03 1.98e+03
  * lat          (feature) float64 0.5 1.5 2.5 3.5 4.5 ... 56.5 57.5 58.5 59.5

In [49]:
train_x_ml_xr

<xarray.DataArray 'rho' (sample: 13600, feature: 4736)>
array([[ 0.27564883,  0.2831928 ,  0.26897484, ...,  0.12592803,
         0.12788226,  0.12482595],
       [ 0.2589989 ,  0.263928  ,  0.2533688 , ...,  0.12034894,
         0.11993926,  0.11608019],
       [ 0.28304878,  0.29282522,  0.26163083, ...,  0.11397284,
         0.11279056,  0.10812949],
       ...,
       [ 0.06474973,  0.06453714,  0.07435823, ..., -0.01673727,
        -0.01668029, -0.01908167],
       [ 0.08787463,  0.09054463,  0.07986625, ..., -0.00557909,
        -0.0095316 , -0.01431126],
       [ 0.10359956,  0.11173593,  0.07619424, ..., -0.01434623,
        -0.01588599, -0.01987674]], dtype=float32)
Coordinates:
    lev          float64 ...
  * sample       (sample) object MultiIndex
  * realization  (sample) object 'lkm0001' 'lkm0001' ... 'lkm0100' 'lkm0100'
  * time         (sample) datetime64[ns] 1870-07-01T06:00:00 ... 2005-07-01T0...
  * feature      (feature) object MultiIndex
  * lon          (feature) float64 -96.0 -96.0 -95.0 -95.0 ... 10.0 10.0 10.0
  * lat          (feature) float64 23.5 24.5 20.5 21.5 ... -7.5 -6.5 -5.5 -4.5

In [52]:
model = MultiOutputRegressor(sklearn.linear_model.Ridge(random_state=123, alpha=0)).fit(train_x_ml_xr, train_y_ml_xr)

In [ ]:
alpha_list = np.logspace(-2,4, 100)

In [ ]:
train_mse_list = []
valid_mse_list = []


for i,alpha in tqdm(enumerate(alpha_list)): 
    sklearn.MultiOutputRegressor()
    model = sklearn.linear_model.Ridge(alpha = alpha)
    model.fit(train_x_ml_xr, train_y_ml_xr)
    
    model_coef_ml_xr = xr.DataArray(model.coef_, dims = ["feature"], coords ={"feature":feature_drop_coords} )
    model_coef_tmp,_ = xr.align(model_coef_ml_xr, feature_coords, join="outer")

    train_y_pred_ml_np = model.predict(train_x_ml_xr)
    valid_y_pred_ml_np = model.predict(valid_x_ml_xr)

    train_y_pred_ml_xr = xr.DataArray(train_y_pred_ml_np, dims= ["sample"], coords = {"sample": train_sample_coords, "alpha":alpha})
    valid_y_pred_ml_xr = xr.DataArray(valid_y_pred_ml_np, dims= ["sample"], coords = {"sample": valid_sample_coords, "alpha":alpha})


    train_mse_tmp = mse_xr(train_y_pred_ml_xr, train_y_true_ml_xr)
    valid_mse_tmp = mse_xr(valid_y_pred_ml_xr, valid_y_true_ml_xr)
    
    model_name = "model_{}.sav".format(i)
    model_file = os.path.join(model_path, model_name)
    joblib.dump(model, model_file)
    

    if i == 0: 
        model_coef = model_coef_tmp.assign_coords({"alpha": alpha})
        train_mse = train_mse_tmp.assign_coords({"alpha":alpha})
        valid_mse = valid_mse_tmp.assign_coords({"alpha":alpha})
        train_pred = train_y_pred_ml_xr.assign_coords({"alpha":alpha})
        valid_pred = valid_y_pred_ml_xr.assign_coords({"alpha":alpha})

    else:
        model_coef = xr.concat([model_coef, model_coef_tmp.assign_coords({"alpha":alpha})], dim="alpha")
        train_mse = xr.concat([train_mse, train_mse_tmp.assign_coords({"alpha":alpha})], dim="alpha")
        valid_mse = xr.concat([valid_mse, valid_mse_tmp.assign_coords({"alpha":alpha})], dim="alpha")
        train_pred = xr.concat([train_pred, train_y_pred_ml_xr.assign_coords({"alpha":alpha})], dim="alpha")
        valid_pred = xr.concat([valid_pred, valid_y_pred_ml_xr.assign_coords({"alpha":alpha})], dim="alpha")
        

    data = xr.merge([
        model_coef.rename("coefficients"),
        train_mse.rename("train_mse"), 
        valid_mse.rename("valid_mse"),
        valid_pred.rename("valid_pred"),
        train_pred.rename("train_pred"),
        ])


0it [00:00, ?it/s]/work/uo1075/u301101/programming/miniconda3/envs/CNN/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.09629e-09): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
1it [00:05,  5.08s/it]/work/uo1075/u301101/programming/miniconda3/envs/CNN/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.38278e-09): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
2it [00:08,  4.12s/it]/work/uo1075/u301101/programming/miniconda3/envs/CNN/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.71316e-09): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
3it [00:13,  4.60s/it]/work/uo1075/u301101/programming/miniconda3/envs/CNN/lib/python3.8/site-packages/sklearn/linear_model/_ridge.